# ETL Portion

### Here, we are utilizing my personal data requests from both Facebook and Instagram, and looking through the messages in order to prepare them for sentiment analysis. The files are saved locally, which is why you will see local directories listed.

# Loading Modules

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import json
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re
import time
import spacy
spacy.cli.download('en_core_web_sm')
from spacy.language import Language
from spacy_language_detection import LanguageDetector

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Data Extraction

### First, we pull the data from the raw directory. Meta gives you a pull consisting of a folder for each conversation. These folders contain json files with the message contents. Longer Facebook Conversations can contain multiple jsons.

#### Instagram:

In [2]:
# Getting folder names for each chat in order to iterate through later

# insert your Instagram handle here:
insta_account = 'REDACTED

parent_directory = r'C:\Users\marco\JUPYTER\TextSentimentAnalysis\instagram-{}\messages\inbox'.format(insta_account)

items = os.listdir(parent_directory)

chats = []

for item in items:
    item_path = os.path.join(parent_directory, item)
    if os.path.isdir(item_path):
        chats.append(item)

In [6]:
# Going through each chat and appending it to a dataframe, creating the baseline dataframe for Instagram

num=0

for chat in chats:
    json_file_path = r'C:\Users\marco\JUPYTER\TextSentimentAnalysis\instagram-{}\messages\inbox\{}\message_1.json'.format(insta_account, chat)
    with open(json_file_path, encoding='utf-8') as json_file:
        data = json.load(json_file)
    if num==0:
        insta_df = pd.DataFrame.from_dict(data['messages'])
        num=1
    df_temp=pd.DataFrame.from_dict(data['messages'])
    insta_df = pd.concat([insta_df, df_temp])

In [9]:
print('We have {} Instagram messages.'.format(len(insta_df)))

We have 69719 Instagram messages.


In [11]:
# Dropping NaN values and utilizing regex to filter through dataframe with non-message content (such as likes or 
# messages containing exclusively hyperlinks)

insta_df.dropna(subset=['content'], inplace=True)

link_pattern = r'^(http://|https://|www\.)\S+$'
insta_df = insta_df[~insta_df['content'].str.match(link_pattern, case=False)]

insta_df = insta_df[~insta_df['content'].str.contains(
    r'(?i)((.+sent an attachment.+)|(.+to your message.+)|(liked a message)|(.+shared a story.+)|\
    (https:\/\/www\.|http:\/\/www\.|https:\/\/|http:\/\/)?[a-zA-Z0-9]{2,}(\.[a-zA-Z0-9]{2,})(\.[a-zA-Z0-9]{2,})?)')]

# Leaving only letters
insta_df['content'] = insta_df['content'].apply(lambda x: ''.join(re.findall(r"[A-Za-zÁÉÍÓÚÜáéíóúü\s]+", x)))

print('We have {} Instagram messages.'.format(len(insta_df)))

C:\Users\marco\AppData\Local\Temp\ipykernel_13888\1395765037.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  insta_df = insta_df[~insta_df['content'].str.contains(


We have 49900 Instagram messages.


In [12]:
# Finding the most common phrases, to verify that no repetitive irrelevant messages remain

phrase_length = 3

# Initialize a Counter to store phrase counts
phrase_counter = Counter()

# Iterating through each row in the dataframe, tokenizing and taking a count
for index, row in insta_df.iterrows():
    text = str(row['content'])
    
    words = word_tokenize(text)

    phrases = [tuple(words[i:i+phrase_length]) for i in range(len(words) - phrase_length + 1)]

    phrase_counter.update(phrases)

# Now printing the most n common phrases
top_n = 50
most_common_phrases = phrase_counter.most_common(top_n)

for phrase, count in most_common_phrases:
    print(f'Phrase: {" ".join(phrase)}, Count: {count}')


Phrase: te voy a, Count: 55
Phrase: que no me, Count: 49
Phrase: verdad es que, Count: 48
Phrase: que no se, Count: 48
Phrase: un par de, Count: 48
Phrase: la verdad es, Count: 41
Phrase: no se si, Count: 40
Phrase: que no te, Count: 38
Phrase: I dont know, Count: 38
Phrase: me voy a, Count: 35
Phrase: que se yo, Count: 34
Phrase: de lo que, Count: 32
Phrase: a mi me, Count: 32
Phrase: es lo que, Count: 32
Phrase: no me acuerdo, Count: 27
Phrase: lo voy a, Count: 26
Phrase: todo lo que, Count: 26
Phrase: que no lo, Count: 26
Phrase: un poco de, Count: 26
Phrase: lo que te, Count: 25
Phrase: que voy a, Count: 24
Phrase: que es un, Count: 24
Phrase: voy a estar, Count: 23
Phrase: te va a, Count: 23
Phrase: a tener que, Count: 23
Phrase: no se que, Count: 23
Phrase: va a ser, Count: 23
Phrase: es algo que, Count: 23
Phrase: I wan na, Count: 22
Phrase: I feel like, Count: 22
Phrase: es que no, Count: 22
Phrase: nada que ver, Count: 22
Phrase: no te voy, Count: 22
Phrase: no voy a, Count: 2

#### Facebook

In [23]:
# Getting folder names for each chat in order to iterate through later

# insert your Facebook handle here:
fb_account = 'marcosgrillo'

parent_directory = r'C:\Users\marco\JUPYTER\TextSentimentAnalysis\facebook-{}\your_activity_across_facebook\messages\inbox'.format(fb_account)

items = os.listdir(parent_directory)

chats = []

for item in items:
    item_path = os.path.join(parent_directory, item)
    if os.path.isdir(item_path):
        chats.append(item)

In [24]:
# Going through each chat and appending it to a dataframe, creating the baseline dataframe for Facebook
# Since Facebook stores chats in multiple jsons for longer conversations, it will be necessary to loop through them
# individually

num=0

for chat in chats:
    chat_file_path = r'C:\Users\marco\JUPYTER\TextSentimentAnalysis\facebook-{}\your_activity_across_facebook\messages\inbox\{}'.format(fb_account, chat)
    chat_list=[]
    
    # Creating a list with all the json files in a single folder
    for filename in os.listdir(chat_file_path):
        if filename.endswith('.json'):
            chat_list.append(filename)
            
    # Compiling it all together, taking into account the different jsons
    for log in chat_list:
        log_file_path = r'C:\Users\marco\JUPYTER\TextSentimentAnalysis\facebook-{}\your_activity_across_facebook\messages\inbox\{}\{}'.format(fb_account, chat, log)
    
        with open(log_file_path, encoding='utf-8') as json_file:
            data = json.load(json_file)
        if num==0:
            facebook_df = pd.DataFrame.from_dict(data['messages'])
            num=1
        df_temp=pd.DataFrame.from_dict(data['messages'])
        facebook_df = pd.concat([facebook_df, df_temp])

In [26]:
print('We have {} Facebook messages.'.format(len(facebook_df)))

We have 269463 Facebook messages.


In [29]:
# Dropping NaN values and utilizing regex to filter through dataframe with non-message content (such as likes or 
# messages containing exclusively hyperlinks), as well as some particularly redundant messages I must have thought
# very funny to send in 2011

facebook_df.dropna(subset=['content'], inplace=True)

link_pattern = r'^(http://|https://|www\.)\S+$'
facebook_df = facebook_df[~facebook_df['content'].str.match(link_pattern, case=False)]

facebook_df = facebook_df[~facebook_df['content'].str.contains(
    r'(?i)((.+formed in Seattle.+)|(.+oO oO oO.+)|(.+Y Y Y.+)|\
    (https:\/\/www\.|http:\/\/www\.|https:\/\/|http:\/\/)?[a-zA-Z0-9]{2,}(\.[a-zA-Z0-9]{2,})(\.[a-zA-Z0-9]{2,})?)')]

# Leaving only letters
facebook_df['content'] = facebook_df['content'].apply(lambda x: ''.join(re.findall(r"[A-Za-zÁÉÍÓÚÜáéíóúü\s]+", x)))

print('We have {} Facebook messages.'.format(len(facebook_df)))

C:\Users\marco\AppData\Local\Temp\ipykernel_13888\985993686.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  facebook_df = facebook_df[~facebook_df['content'].str.contains(


We have 260832 Facebook messages.


In [30]:
# Finding the most common phrases, to verify that no repetitive irrelevant messages remain

phrase_length = 3

# Initializing a Counter to store phrase counts
phrase_counter = Counter()

# Iterating through each row in the dataframe, tokenizing and taking a count
for index, row in facebook_df.iterrows():
    text = str(row['content'])
    
    words = word_tokenize(text)

    phrases = [tuple(words[i:i+phrase_length]) for i in range(len(words) - phrase_length + 1)]

    phrase_counter.update(phrases)

# Now printing the most n common phrases (pardon the swear words)
top_n = 50
most_common_phrases = phrase_counter.most_common(top_n)

for phrase, count in most_common_phrases:
    print(f'Phrase: {" ".join(phrase)}, Count: {count}')

Phrase: me voy a, Count: 1312
Phrase: left the group, Count: 516
Phrase: que se yo, Count: 467
Phrase: to the group, Count: 292
Phrase: a lo de, Count: 289
Phrase: te voy a, Count: 286
Phrase: voy a hacer, Count: 268
Phrase: no se si, Count: 255
Phrase: me tengo que, Count: 252
Phrase: un par de, Count: 248
Phrase: lo voy a, Count: 236
Phrase: de lo que, Count: 235
Phrase: que no me, Count: 234
Phrase: va a ser, Count: 231
Phrase: es lo que, Count: 231
Phrase: en lo de, Count: 229
Phrase: tengo que ir, Count: 223
Phrase: a mi me, Count: 212
Phrase: lo que me, Count: 204
Phrase: no voy a, Count: 198
Phrase: no se que, Count: 198
Phrase: que no se, Count: 197
Phrase: ni en pedo, Count: 196
Phrase: voy a dormir, Count: 187
Phrase: que voy a, Count: 186
Phrase: no creo que, Count: 185
Phrase: lo que te, Count: 176
Phrase: voy a comer, Count: 175
Phrase: me dijo que, Count: 174
Phrase: a la noche, Count: 174
Phrase: che me voy, Count: 173
Phrase: que ir a, Count: 172
Phrase: a tener que, Co

# Merging

In [31]:
# Merging the Facebook and Instagram dataframes, keeping only sender, timestamp and content, since it's the only part 
# we really care about

merged_df = pd.concat([facebook_df[['sender_name', 'timestamp_ms', 'content']],
                       insta_df[['sender_name', 'timestamp_ms', 'content']]])

# Eliminating excessively long messages

merged_df = merged_df[merged_df['content'].str.len() < 15000]

In [32]:
# Shifting timestamp from ms to datetime

merged_df['timestamp_ms'] = pd.to_datetime(merged_df['timestamp_ms'], unit='ms')

# Shifting time to my local timezone from UTC

merged_df['timestamp_ms'] = merged_df['timestamp_ms'] - pd.Timedelta(hours=4)

In [33]:
# Pickling to be able to quickly pickup where I left off

merged_df.to_pickle('Extracted_Transformed.pkl')

# Language Identification

In [ ]:
# Utilizing spacy to identify the language of each text message. This takes a while to run.

merged_df = merged_df.reset_index(drop=True)

merged_df['Language'] = None 
merged_df['Confidence'] = None

def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)  # The answer to everything


nlp_model = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)

# Just for getting estimates for remaining time
start = time.time()

for row in range(len(merged_df)):
    
    message = nlp_model(merged_df.loc[row, 'content'])
    language = message._.language
    
    # Getting measures for the language and the answer confidence
    merged_df.loc[row, 'Language'] = language['language']
    merged_df.loc[row, 'Confidence'] = language['score']
    
    # Printing progress estimates periodically
    if (row+1)%1000==0:
        elapsed = time.time()-start
        percent_done = (row/len(merged_df))*100
        time_left = (elapsed/(row/len(merged_df)) - elapsed)/60
        print('{}th row. {} % done. Time left: {} minutes'.format(row+1, percent_done, time_left))

In [34]:
# Pickling (again!). See you in the sentiment analysis portion!

merged_df.to_pickle('Extracted_Transformed_LangTagged.pkl')